In [12]:
import utilities
import matplotlib.pyplot as plt
import numpy as np
import AdaBoost
import KernelPerceptron
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import ImageGrid

Populating the interactive namespace from numpy and matplotlib


In [13]:
height = width = 16
class_number = 10
N = 20

#train_type = 'one_Vs_all'
train_type = 'one_Vs_one'

#algorithm = 'AdaBoost'
algorithm = 'KernelPerceptron'
#algorithm = 'SVM'


#kernel_type = 'gaussian'
kernel_type = 'polynomial'
#kernel_type = ' '

parameters = [1,2,3,4,5,6,7]

#parameters = [0.01,0.1,0.6,1,2,5]
#parameters = [0.1,0.2,0.3,0.5]
#parameters = [0.08,0.09,0.1,0.15,0.25]

#parameters = [30,40,50,60,70]
#parameters = [60,80,100]

#parameters = [0.1,0.5,1,1.5,2,3,4,5,6]
#parameters = [4,5,6]

EXPERIMENT_TYPE = 1

In [14]:
train_data = 'Datasets/full_dataset.dat'
data,labels = utilities.read_data(train_data,width,height)
print(data.shape)
print(labels.shape)

(9297, 256)
(9297,)


In [15]:
#data = data[0:1000]
#labels = labels[0:1000]

In [16]:
#utilities.get_average_confusions(confusion_error_list)
if(EXPERIMENT_TYPE == 1):
    
    ## N runs for random train test configurations
    results = np.zeros((N,len(parameters)))
    
    #confusion_error_list = []

    for run_index in range(0,N):
    
        train_data,test_data,train_labels,test_labels = utilities.data_split(data,labels,0.2)
    
        data_per_class_dictionary = utilities.collect_each_class_images(train_data,train_labels,class_number)
        pair_datasets = utilities.create_1_VS_1_dataset(data_per_class_dictionary)
        #print("Paired classifiers number: " + str(len(pair_datasets)))

        #print(train_data.shape)
        #print(test_data.shape)
        #print(train_labels.shape)
        #print(test_labels.shape)
        for i in range(0,len(parameters)):
        
            param = parameters[i]
        
            param_list = []
            param_list = param_list + [kernel_type]
            param_list  = param_list + [float(param)]
            print(train_type + " training for algorithm " + algorithm + " with params " + str(param_list) +  " and random split " + str(run_index+1))
        
            models_dict = None
            pair_models = None
            if(train_type == 'one_Vs_all'):
                models_dict = utilities.one_VS_all_training(data_per_class_dictionary,class_number,algorithm,param_list)
            else:
                pair_models = utilities.one_vs_one_training(pair_datasets,algorithm,param_list)
         
            test_predictions = None
            if(train_type == 'one_Vs_all'):
                test_predictions,test_confidence,average_confidence = utilities.one_VS_all_testing(algorithm,models_dict,test_data,test_labels)
            else:
                test_predictions = utilities.one_vs_one_testing(algorithm,pair_models,test_data,class_number)
           
            results[run_index][i] = utilities.calculate_accuracy(test_predictions,test_labels)
            
            #confusion_error = utilities.confusion_matrix(test_predictions,test_labels)
            
            #confusion_error_list.append(confusion_error)
            
            
    for i in range(results.T.shape[0]):
        print("Mean error and std for parameter: " + str(parameters[i]) + " is " + str(np.mean(results.T[i])) + " - " + str(np.std(results.T[i])))
        
    #utilities.get_average_confusions(confusion_error_list)

one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 1.0] and random split 1
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 2.0] and random split 1
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 3.0] and random split 1
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 4.0] and random split 1
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 5.0] and random split 1
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 6.0] and random split 1
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 7.0] and random split 1
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 1.0] and random split 2
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 2.0] and random split 2
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 3.0]

one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 5.0] and random split 12
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 6.0] and random split 12
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 7.0] and random split 12
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 1.0] and random split 13
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 2.0] and random split 13
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 3.0] and random split 13
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 4.0] and random split 13
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 5.0] and random split 13
one_Vs_one training for algorithm KernelPerceptron with params ['polynomial', 6.0] and random split 13
one_Vs_one training for algorithm KernelPerceptron with params ['polynomi

In [17]:
if(EXPERIMENT_TYPE == 2):
    
    results = []
    
    confusion_error_list = []
    
    for run_index in range(0,N):
        
        print("------------------------------------------------")
        
        ## make a random split (80%-20%) on the dataset
        train_data,final_test_data,train_labels,final_test_labels = utilities.data_split(data,labels,0.2)
        
        cross_valid_results = np.zeros((5,len(parameters)))
    
        ## then perform 5-k cross validation to find the best parameter on it
        for k in range(5):
            ## given a validation round split the training data into training and validation data of the round
            validation_indexes,training_indexes = utilities.cross_validation(k,len(train_data),5)
            round_train_data,round_train_labels,round_valid_data,round_valid_labels = utilities.get_train_test_set(validation_indexes,training_indexes,train_data,train_labels)
            
            data_per_class_dictionary = utilities.collect_each_class_images(round_train_data,round_train_labels,class_number)
            pair_datasets = utilities.create_1_VS_1_dataset(data_per_class_dictionary)
        
            ## for every cross validation round check all the parameters
            for i in range(0,len(parameters)):
                param = parameters[i]
                param_list = []
                param_list = param_list + [kernel_type]
                param_list  = param_list + [float(param)]
                #print(train_type + " training for algorithm " + algorithm + " with params " + str(param_list) +  " random split " + str(run_index+1) + " and cross validation round: " + str(k + 1))
        
                models_dict = None
                pair_models = None
                if(train_type == 'one_Vs_all'):
                    models_dict = utilities.one_VS_all_training(data_per_class_dictionary,class_number,algorithm,param_list)
                else:
                    pair_models = utilities.one_vs_one_training(pair_datasets,algorithm,param_list)
         
                test_predictions = None
                if(train_type == 'one_Vs_all'):
                    test_predictions,test_confidence,average_confidence = utilities.one_VS_all_testing(algorithm,models_dict,round_valid_data,round_valid_labels)
                else:
                    test_predictions = utilities.one_vs_one_testing(algorithm,pair_models,round_valid_data,class_number)
            
                cross_valid_results[k][i] = utilities.calculate_accuracy(test_predictions,round_valid_labels)
    
        best_param_list = []
        ## find the best parameter of this validation round
        ## computing the average error for each parameter for each split
        cross_valid_results = cross_valid_results.T
        for i in range(cross_valid_results.shape[0]):
            best_param_list.append((i,np.mean(cross_valid_results[i])))
        
        ## get the best result and the parameter for which the result achieved
        best_param_list = sorted(best_param_list, key=lambda tup: tup[1],reverse = False)
        
        best_param = parameters[best_param_list[0][0]]
        
        #print("Best Parameter for run round " + str(run_index+1) + " is " + str(best_param) + " with error: " + str(best_param_list[0][1]))
        
        ## now retrain in full 80% dataset and test for the remaining hidden 20% test set
        data_per_class_dictionary = utilities.collect_each_class_images(train_data,train_labels,class_number)
        pair_datasets = utilities.create_1_VS_1_dataset(data_per_class_dictionary)
        
        param = best_param
        param_list = []
        param_list = param_list + [kernel_type]
        param_list  = param_list + [float(param)]
        print(train_type + " training for algorithm " + algorithm + " with params " + str(param_list) + " for the full dataset")
        
        models_dict = None
        pair_models = None
        if(train_type == 'one_Vs_all'):
            models_dict = utilities.one_VS_all_training(data_per_class_dictionary,class_number,algorithm,param_list)
        else:
            pair_models = utilities.one_vs_one_training(pair_datasets,algorithm,param_list)
         
        ## test on the remaining 20% which was hidden through the cross validation
        test_predictions = None
        if(train_type == 'one_Vs_all'):
            test_predictions,test_confidence,average_confidence = utilities.one_VS_all_testing(algorithm,models_dict,final_test_data,final_test_labels)
        else:
            test_predictions = utilities.one_vs_one_testing(algorithm,pair_models,final_test_data,class_number)
         
        accuracy = utilities.calculate_accuracy(test_predictions,final_test_labels)
        results.append((accuracy,best_param))
        
        confusion_error_list.append(utilities.confusion_matrix(test_predictions,final_test_labels))
        
    print(results)
    
    error = []
    param = []
    for (e,p) in results:
        error.append(e)
        param.append(p)
        
    print("--------")

        
    print("Mean error: " + str(np.mean(error)))
    print("Std error: " + str(np.std(error)))
    print("Mean param: " + str(np.mean(param)))
    print("Std param: " + str(np.std(param)))

    
    print("--------")
    
    utilities.get_average_confusions(confusion_error_list)
        

In [18]:
if(EXPERIMENT_TYPE == 3):

    data_per_class_dictionary = utilities.collect_each_class_images(data,labels,class_number)
    param_list = []
    param_list = param_list + [kernel_type]
    param_list  = param_list + [float(parameters[4])]
    models_dict = utilities.one_VS_all_training(data_per_class_dictionary,class_number,algorithm,param_list)
    predictions,confidence,average_confidence = utilities.one_VS_all_testing(algorithm,models_dict,data,labels)


    hard_items_indexes_1,hard_items_indexes_2 = utilities.get_hardest_to_predict_items(confidence,average_confidence,predictions,labels)
    #print(hard_items_indexes_1)
    #print(hard_items_indexes_2)
    hard_images = []
    hard_items_indexes = hard_items_indexes_1
    for index in hard_items_indexes:
        #imgplot = plt.imshow(data[index].reshape((16,16)))
        #plt.show()
        hard_images.append(data[index].reshape((16,16)))
        
    fig = plt.figure(figsize=(8., 8.))
    grid = ImageGrid(fig, 111,nrows_ncols=(2,5),axes_pad=0.1)

    for ax, im in zip(grid,hard_images):
        ax.imshow(im)

    plt.show()